In [1]:
from viz_mekko import *

In [2]:
df = pd.read_csv('athlete_events.csv')

In [3]:
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [6]:
def plot_marimekko(dataframe, country):
    unique_years = dataframe['Year'].unique()
    # Filter data for the specified country
    df_country = dataframe[dataframe['NOC'] == country]
    df_country.loc[:, 'Medal'] = df_country['Medal'].replace({'Silver': 'Prata', 'Gold': 'Ouro'})


    # Group by year and medal type and count the number of medals
    medal_counts = df_country.groupby(['Year', 'Medal']).size().reset_index(name='count')
    medal_counts = fill_in_years(medal_counts, unique_years)
    # Pivot the dataframe to have medal types as columns
    medal_pivot = medal_counts.pivot(index='Year', columns='Medal', values='count').fillna(0)

    # Calculate total medals per year and proportions for each medal type
    medal_pivot['total'] = medal_pivot.sum(axis=1)
    for medal in ['Ouro', 'Prata', 'Bronze']:
        medal_pivot[f'Medalhas de {medal}'] = medal_pivot[medal] / medal_pivot['total']

    # Normalize widths based on the total medals to fit the graph size
    total_medals = medal_pivot['total'].sum()
    medal_pivot['width'] = medal_pivot['total'] / total_medals

    # Compute the x positions for bars to ensure they do not overlap
    x_positions = [0]  # Starting position for the first bar
    for width in medal_pivot['width'][:-1]:
        x_positions.append(x_positions[-1] + width + 0.05)  # Add small gap between bars
    medal_pivot['x'] = x_positions

    # Create the Marimekko chart with adjusted x-axis
    fig = go.Figure()

    # Add bronze trace first
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Bronze'],
        width=medal_pivot['width'],
        base=0,
        marker=dict(color='#cd7f32'),
        name='Bronze',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Bronzes: %{y:.2f}<extra></extra>',
    ))

    # Add silver trace second
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Prata'],
        width=medal_pivot['width'],
        base=medal_pivot['Medalhas de Bronze'],
        marker=dict(color='#c0c0c0'),
        name='Prata',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Pratas: %{y:.2f}<extra></extra>',
    ))

    # Add gold trace last
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Ouro'],
        width=medal_pivot['width'],
        base=medal_pivot['Medalhas de Bronze'] + medal_pivot['Medalhas de Prata'],
        marker=dict(color='#ffd700'),
        name='Ouro',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Ouros: %{y:.2f}<extra></extra>',
    ))

    # Add transparent phantom bars where medal count is 0
    for year in medal_pivot.index:
        for medal in ['Ouro', 'Prata', 'Bronze']:
            if medal_pivot.at[year, f'Medalhas de {medal}'] == 0:
                fig.add_trace(go.Bar(
                    x=[medal_pivot.at[year, 'x']],
                    y=[0],  # Phantom bar at y=0
                    width=[medal_pivot.at[year, 'width']],
                    base=0,
                    marker=dict(color='rgba(0,0,0,0)'),  # Transparent marker
                    showlegend=False,  # Do not show in legend
                ))

    # Update layout for the Marimekko chart
    fig.update_layout(
        title=f'Proporção de Medalhas - {country}',
        barmode='stack',
        plot_bgcolor='white',  # Set the background color to white
        xaxis=dict(
            title='Year',
            tickmode='array',
            tickvals=medal_pivot['x'],
            ticktext=medal_pivot.index.astype(str),
            showgrid=False,  # Hide vertical grid lines
        ),
        yaxis=dict(
            title='Proporção de Medalhas',
            tickformat='.0%',
            gridcolor='lightgray',  # Set horizontal grid lines to light gray
        ),
    )

    fig.show()


In [7]:
plot_marimekko(df, 'BRA')


In [9]:
def plot_marimekko(dataframe, country):
    unique_years = dataframe['Year'].unique()
    # Filter data for the specified country
    df_country = dataframe[dataframe['NOC'] == country]
    df_country.loc[:, 'Medal'] = df_country['Medal'].replace({'Silver': 'Prata', 'Gold': 'Ouro'})


    # Group by year and medal type and count the number of medals
    medal_counts = df_country.groupby(['Year', 'Medal']).size().reset_index(name='count')
    medal_counts = fill_in_years(medal_counts, unique_years)
    # Pivot the dataframe to have medal types as columns
    medal_pivot = medal_counts.pivot(index='Year', columns='Medal', values='count').fillna(0)

    # Calculate total medals per year and proportions for each medal type
    medal_pivot['total'] = medal_pivot.sum(axis=1)
    for medal in ['Ouro', 'Prata', 'Bronze']:
        medal_pivot[f'Medalhas de {medal}'] = medal_pivot[medal] / medal_pivot['total']

    # Normalize widths based on the total medals to fit the graph size
    total_medals = medal_pivot['total'].sum()
    medal_pivot['width'] = medal_pivot['total'] / total_medals

    # Compute the x positions for bars to ensure they are properly centered
    bar_width = 0.25  # Width of each bar
    padding = 0.05  # Padding between bars
    x_positions = [bar_width / 2]  # Starting position for the first bar
    for width in medal_pivot['width'][:-1]:
        x_positions.append(x_positions[-1] + width + padding)  # Add padding between bars
    medal_pivot['x'] = x_positions

    # Create the Marimekko chart with adjusted x-axis
    fig = go.Figure()

    # Add bronze trace first
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Bronze'],
        width=medal_pivot['width'],
        base=0,
        marker=dict(color='#cd7f32'),
        name='Bronze',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Bronzes: %{y:.2f}<extra></extra>',
    ))

    # Add silver trace second
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Prata'],
        width=medal_pivot['width'],
        base=medal_pivot['Medalhas de Bronze'],
        marker=dict(color='#c0c0c0'),
        name='Prata',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Pratas: %{y:.2f}<extra></extra>',
    ))

    # Add gold trace last
    fig.add_trace(go.Bar(
        x=medal_pivot['x'],
        y=medal_pivot['Medalhas de Ouro'],
        width=medal_pivot['width'],
        base=medal_pivot['Medalhas de Bronze'] + medal_pivot['Medalhas de Prata'],
        marker=dict(color='#ffd700'),
        name='Ouro',
        customdata=medal_pivot['total'],
        hovertemplate='Year: %{x}<br>Total de Medalhas: %{customdata}<br>Proporção de Ouros: %{y:.2f}<extra></extra>',
    ))

    # Add transparent phantom bars where medal count is 0
    for year in medal_pivot.index:
        for medal in ['Ouro', 'Prata', 'Bronze']:
            if medal_pivot.at[year, f'Medalhas de {medal}'] == 0:
                fig.add_trace(go.Bar(
                    x=[medal_pivot.at[year, 'x']],
                    y=[0],  # Phantom bar at y=0
                    width=[medal_pivot.at[year, 'width']],
                    base=0,
                    marker=dict(color='rgba(0,0,0,0)'),  # Transparent marker
                    showlegend=False,  # Do not show in legend
                ))

    # Update layout for the Marimekko chart
    fig.update_layout(
        title=f'Proporção de Medalhas - {country}',
        barmode='stack',
        plot_bgcolor='white',  # Set the background color to white
        xaxis=dict(
            title='Year',
            tickmode='array',
            tickvals=medal_pivot['x'],
            ticktext=medal_pivot.index.astype(str),
            showgrid=False,  # Hide vertical grid lines
        ),
        yaxis=dict(
            title='Proporção de Medalhas',
            tickformat='.0%',
            gridcolor='lightgray',  # Set horizontal grid lines to light gray
        ),
    )

    fig.show()

plot_marimekko(df, 'BRA')



In [5]:
country = 'BRA'